In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [8]:
# Dummy dataset with sequence and structure features
class ProteinDataset(Dataset):
    def __init__(self, num_samples=1000, seq_length=128, feature_dim=64):
        self.seq_features = np.random.rand(num_samples, seq_length, feature_dim)
        self.struct_features = np.random.rand(num_samples, seq_length, feature_dim)
        self.labels = np.random.randint(0, 2, (num_samples, seq_length))  # Binary labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.seq_features[idx], dtype=torch.float32),
            torch.tensor(self.struct_features[idx], dtype=torch.float32),
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

In [9]:
# Co-Attention Mechanism for Multi-View Learning
class CoAttention(nn.Module):
    def __init__(self, feature_dim):
        super(CoAttention, self).__init__()
        self.seq_fc = nn.Linear(feature_dim, feature_dim)
        self.struct_fc = nn.Linear(feature_dim, feature_dim)
        self.attn = nn.Linear(feature_dim, 1)

    def forward(self, seq_features, struct_features):
        seq_proj = torch.tanh(self.seq_fc(seq_features))
        struct_proj = torch.tanh(self.struct_fc(struct_features))
        attn_weights = torch.softmax(self.attn(seq_proj + struct_proj), dim=1)
        co_attended = attn_weights * seq_features + (1 - attn_weights) * struct_features
        return co_attended


In [10]:
# Prediction Model with Multi-View Learning
class ATPBindingPredictor(nn.Module):
    def __init__(self, seq_length=128, feature_dim=64, hidden_dim=128):
        super(ATPBindingPredictor, self).__init__()
        self.co_attention = CoAttention(feature_dim)
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, 2)  # Binary classification

    def forward(self, seq_features, struct_features):
        co_attended = self.co_attention(seq_features, struct_features)
        lstm_out, _ = self.lstm(co_attended)
        logits = self.fc(lstm_out)
        return logits

In [11]:
# Training the Model
def train_model(model, train_loader, epochs=5, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for seq_features, struct_features, labels in train_loader:
            seq_features, struct_features, labels = seq_features.to(device), struct_features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(seq_features, struct_features)
            loss = criterion(outputs.view(-1, 2), labels.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


In [ ]:
# Main Execution
if __name__ == "__main__":
    dataset = ProteinDataset()
    train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
    model = ATPBindingPredictor()
    train_model(model, train_loader)

Epoch 1, Loss: 0.6938
